In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
#from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
import calendar
import time
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
import re
from sklearn.metrics import mean_squared_error

In [2]:
train_data = pd.read_csv("duom/train_data_1.csv")
test_data = pd.read_csv("duom/test_data_1.csv")

In [3]:
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
cols = ['term', 'credit_score', 'loan_purpose', 'home_ownership']
for col in cols:
    train_data[col] = le1.fit_transform(train_data[col])
    test_data[col] = le2.fit_transform(test_data[col])
    print(le1.classes_)
    print(le2.classes_)

y = train_data['y']
x = train_data.drop(['y', 'id'], axis=1)

['long' 'short']
['long' 'short']
['fair' 'good' 'unknown' 'very_good']
['fair' 'good' 'unknown' 'very_good']
['business_loan' 'buy_a_car' 'buy_house' 'debt_consolidation'
 'educational_expenses' 'home_improvements' 'major_purchase'
 'medical_bills' 'moving' 'other' 'small_business' 'take_a_trip'
 'vacation' 'wedding']
['business_loan' 'buy_a_car' 'buy_house' 'debt_consolidation'
 'educational_expenses' 'home_improvements' 'major_purchase'
 'medical_bills' 'moving' 'other' 'small_business' 'take_a_trip'
 'vacation' 'wedding']
['mortgage' 'own' 'rent']
['mortgage' 'own' 'rent']


In [4]:
#
# Create training and validation sets
#
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#
# Create the LightGBM data containers
#
lgb_train = lgb.Dataset(x_train, label=y_train)
lgb_eval = lgb.Dataset(x_test, label=y_test, reference=lgb_train)

In [5]:
"""'metric': 'auc',
    'objective': 'binary',
    'learning_rate': 0.018782604994529304, 
    'max_depth': 38, 
    'num_leaves': 24708, 
    'feature_fraction': 0.965292624452857, 
    'subsample': 0.721752904462013
params = {    
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'lambda_l1': 0.00004, 
    'lambda_l2': 0.004, 
    'num_leaves': 40000, 
    'feature_fraction': 0.995, 
    'bagging_fraction': 0.974, 
    'bagging_freq': 1, 
    'min_child_samples': 14,
    'max_bin': 6000,
    'is_unbalace': True
}

print('Starting training...')
# train
model = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=5)])

print('Starting predicting...')
# predict
y_pred = model.predict(x_test, num_iteration=model.best_iteration)
# eval
rmse_test = mean_squared_error(y_test, y_pred) ** 0.5
print(f'The RMSE of prediction is: {rmse_test}')

preds = model.predict(x_test)
pred_labels = np.rint(preds)
accuracy = metrics.accuracy_score(y_test, pred_labels)
print(f'The ACU of prediction is: {accuracy}')
#0.8707655164909319
#0.8764131445695981"""

"'metric': 'auc',\n    'objective': 'binary',\n    'learning_rate': 0.018782604994529304, \n    'max_depth': 38, \n    'num_leaves': 24708, \n    'feature_fraction': 0.965292624452857, \n    'subsample': 0.721752904462013\nparams = {    \n    'boosting_type': 'gbdt',\n    'objective': 'regression',\n    'lambda_l1': 0.00004, \n    'lambda_l2': 0.004, \n    'num_leaves': 40000, \n    'feature_fraction': 0.995, \n    'bagging_fraction': 0.974, \n    'bagging_freq': 1, \n    'min_child_samples': 14,\n    'max_bin': 6000,\n    'is_unbalace': True\n}\n\nprint('Starting training...')\n# train\nmodel = lgb.train(params,\n                lgb_train,\n                num_boost_round=20,\n                valid_sets=lgb_eval,\n                callbacks=[lgb.early_stopping(stopping_rounds=5)])\n\nprint('Starting predicting...')\n# predict\ny_pred = model.predict(x_test, num_iteration=model.best_iteration)\n# eval\nrmse_test = mean_squared_error(y_test, y_pred) ** 0.5\nprint(f'The RMSE of prediction

In [6]:
SEARCH_PARAMS = {'learning_rate': 0.018782604994529304, 
                'max_depth': 38, 
                'num_leaves': 24708, 
                'feature_fraction': 0.965292624452857, 
                'subsample': 0.721752904462013}

def train_evaluate(search_params):
    params = {'objective': 'binary',
              'metric': 'auc',
              **search_params}

    model = lgb.train(params, lgb_train,
                      num_boost_round=500,
                      #early_stopping_rounds=10,
                      callbacks=[lgb.early_stopping(stopping_rounds=50)],
                      valid_sets=lgb_eval,
                      valid_names='valid'
                     )
    
    score = model.best_score['valid']['auc']
    return score, model

if __name__ == '__main__':
    score, model = train_evaluate(SEARCH_PARAMS)
    print('validation AUC:', score)

[LightGBM] [Info] Number of positive: 1263373, number of negative: 4155746
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.227856 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1541
[LightGBM] [Info] Number of data points in the train set: 5419119, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.233133 -> initscore=-1.190707
[LightGBM] [Info] Start training from score -1.190707
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[246]	valid's auc: 0.905793
validation AUC: 0.9057928334465929


In [7]:
ids = test_data['id'].values
test_data = test_data.drop(['id'], axis=1)

In [8]:
x = test_data.values
y = model.predict(x)
output = pd.DataFrame({'id': ids, 'target': y})
output.head(15)

,id,target
0,1,0.107575
1,2,0.990458
2,3,0.072420
3,4,0.159505
4,5,0.990559
5,6,0.113315
6,7,0.002819
7,8,0.294503
8,9,0.002453
9,10,0.684629


In [10]:
output.to_csv("submission_1_GBM.csv", index=False)

In [17]:
model.save_model('model_1.txt')

In [9]:
"""import skopt

SPACE = [
    skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
    skopt.space.Integer(1, 50, name='max_depth'),
    skopt.space.Integer(1000, 40000, name='num_leaves'),
    skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),
    skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')]
@skopt.utils.use_named_args(SPACE)
def objective(**params):
    return -1.0 * train_evaluate(params)
results = skopt.forest_minimize(objective, SPACE, n_calls=30, n_random_starts=10)
best_auc = -1.0 * results.fun
best_params = results.x

print('best result: ', best_auc)
print('best parameters: ', best_params)"""

"import skopt\n\nSPACE = [\n    skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),\n    skopt.space.Integer(1, 50, name='max_depth'),\n    skopt.space.Integer(1000, 40000, name='num_leaves'),\n    skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),\n    skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform')]\n@skopt.utils.use_named_args(SPACE)\ndef objective(**params):\n    return -1.0 * train_evaluate(params)\nresults = skopt.forest_minimize(objective, SPACE, n_calls=30, n_random_starts=10)\nbest_auc = -1.0 * results.fun\nbest_params = results.x\n\nprint('best result: ', best_auc)\nprint('best parameters: ', best_params)"